# skinfood

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def seeAllProducts():
    try:
        driver.find_element_by_xpath('//ul[@id="category"]/li[@class="first"]').click() # 전체상품 보기
    except:
        driver.find_element_by_xpath('//ul[@id="category"]/li[@class="first on"]').click() # 전체상품 보기


In [4]:
def clickSeeMoreButton():
    while True:
        try:
            seeMoreButton = driver.find_element_by_id('moreA') 
            driver.execute_script('arguments[0].scrollIntoView(true);', seeMoreButton)
            seeMoreButton.click()
            time.sleep(1)
        except:
            break


In [5]:
def getItemList():
    html = driver.page_source
    soup = bs(html,'html.parser')
    items = soup.find('ul',{'id':'tbodyList'})
    

    itemList = []
    for i, item in enumerate(items):
        try:
            itemList.append(url_home + item.a['href'])
        except:
            pass
    return itemList

In [23]:
def getItem():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    item={}
    # name
    for i, name in enumerate(soup.find('h3',{'class':'productTitle'})):
        if i==2:
            item['name'] = name.strip()
            break

    # image
    item['image'] = 'http:'+soup.find('img',{'id':'productImg'})['src']

    # category
    categoryList = soup.find('div',{'class':'location'}).h2
    category=''
    for c in categoryList:
        if category:
            category += c.get_text().strip()
        else:
            category += c.get_text().strip()
            category += ' > '
    item['category']=category
    # volume
    item['volume'] = soup.find('li',{'class':'capacity'}).get_text().strip()

    # salePrice, originalPrice
    price = soup.find('span',{'id':'selPr'})
    item['originalPrice'] = ''
    for p in price:
        try:
            price_class = p['class']
            if price_class == ['text-del']:
                item['originalPrice'] = getNumber(p.get_text())
            elif price_class == ['price-pic']:
                item['salePrice'] = getNumber(p.get_text())
        except:
            continue
    if item['originalPrice']=='':
        item['originalPrice']=item['salePrice']

    # brand
    item['brand'] = '스킨푸드'

    # url 
    item['url'] = driver.current_url

    # color
    # 품절, 수량
    colors=[]
    items=[]
    colorList = soup.find('ul',{'class':'heapOptions'})
    if colorList:
        for i, color in enumerate(colorList):
            if i==0:
                continue
            color =  color.a['title'].strip()
            '''
            if '[' in color:
                color = color[:color.find('[')].strip()
            '''
            if '수량' in color:
                color = color[:color.find('수량')].strip()
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            items.append(item_copy)
    else:
        item['color']=''
        items.append(item)

    display(items)
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [24]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'http://www.theskinfood.com'
url_products = 'http://www.theskinfood.com/goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B'


In [25]:
start_time = time.time() 
#-------------------------------------------------------------#
driver.get(url_products)
seeAllProducts()

clickSeeMoreButton()
result = []
itemList = getItemList()

for item in itemList:
    driver.get(item)
    result += getItem()
#-------------------------------------------------------------#
print("--- %0.2f seconds ---" %(time.time() - start_time))

[{'name': '흑석류 에멀전',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/61f546bf-c28d-435e-a1ff-276f3f24988a.jpg',
  'category': 'SKIN CARE > 에멀전',
  'volume': '160ml',
  'originalPrice': 23000,
  'salePrice': 18400,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=70202',
  'color': ''}]

[{'name': '흑석류 토너',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/9209c40b-2b3d-486d-b668-5bc0bb285791.jpg',
  'category': 'SKIN CARE > 토너',
  'volume': '160ml',
  'originalPrice': 22000,
  'salePrice': 17600,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=70201',
  'color': ''}]

[{'name': '골드 캐비어 에멀전',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/68f5be5f-1b80-4d80-9efe-17a9d21f8077.jpg',
  'category': 'SKIN CARE > 에멀전',
  'volume': '160ml',
  'originalPrice': 25000,
  'salePrice': 20000,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=70102',
  'color': ''}]

[{'name': '골드 캐비어 토너',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/100bc671-12c8-4f1a-a759-909925ad09ff.jpg',
  'category': 'SKIN CARE > 토너',
  'volume': '160ml',
  'originalPrice': 24000,
  'salePrice': 19200,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=70101',
  'color': ''}]

[{'name': '로열허니 프로폴리스 인리치 마스킹 시트',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/1ec66c18-cac1-4aa8-ab09-fbb434bba9be.jpg',
  'category': 'SKIN CARE > 팩/마스크',
  'volume': '18g',
  'originalPrice': 5000,
  'salePrice': 5000,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=3367',
  'color': ''}]

...

[{'name': '토마토 선 크림 SPF36 PA++',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/7b4145aa-ddf7-4fad-81a2-84d69728cf96.jpg',
  'category': 'SKIN CARE > 선케어',
  'volume': '50ml',
  'originalPrice': 7900,
  'salePrice': 7900,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=613',
  'color': ''}]

[{'name': '블랙슈가 마스크 워시오프',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/7a7a762f-b8a5-4549-b3a2-e31e516a9508.jpg',
  'category': 'SKIN CARE > 팩/마스크',
  'volume': '100g',
  'originalPrice': 9900,
  'salePrice': 9900,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=62',
  'color': ''}]

[{'name': '상추오이 워터 미스트',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/700-1_1.jpg',
  'category': 'SKIN CARE > 미스트',
  'volume': '50ml',
  'originalPrice': 7700,
  'salePrice': 7700,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=700-1',
  'color': ''}]

[{'name': '블랙슈가 스트로베리 마스크 워시오프',
  'image': 'http://www.theskinfood.com/custom/goods/webImg/62-13_1.jpg',
  'category': 'SKIN CARE > 팩/마스크',
  'volume': '100g',
  'originalPrice': 10900,
  'salePrice': 10900,
  'brand': '스킨푸드',
  'url': 'http://www.theskinfood.com/goods/goodsDetail.do?goodsCd=62-13',
  'color': ''}]

--- 295.26 seconds ---


In [ ]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'skinfood.json')